# Доклад "Команды и оркестрация: Команда, Наблюдатель, Посредник, Цепочка обязанностей
### Command and Orchestration Patterns: Command, Observer, Mediator, Chain of Responsibility



Ахмадимасуд Мобина, Б83

---

## Introduction — What are Orchestration Patterns? / Введение — Что такое Паттерны оркестровки?

**English**

Orchestration patterns are structural and behavioral design patterns that help organize how objects and components interact to achieve complex workflows. They focus on *who controls* operations, *how responsibilities are delegated*, and *how communication is coordinated* between parts of a system. The four patterns in this notebook demonstrate common approaches:

- **Command**: encapsulates operations as objects (useful for undo, queuing, and logging).
- **Observer**: allows objects to subscribe to and receive updates from another object (publish–subscribe style).
- **Mediator**: centralizes complex communication between components in a single place to reduce coupling.
- **Chain of Responsibility**: passes requests along a chain of handlers until one handles it.

**Russian (Русский)**

Паттерны оркестровки — это структурные и поведенческие шаблоны проектирования, которые помогают организовать взаимодействие объектов и компонентов для выполнения сложных рабочих процессов. Они определяют, **кто управляет** операциями, **как делегируются обязанности** и **как координируется общение** между частями системы. Четыре паттерна в этом блокноте показывают распространённые подходы:

- **Command (Команда)**: инкапсулирует операции как объекты (полезно для отмены, очередей и логирования).
- **Observer (Наблюдатель)**: позволяет объектам подписываться и получать обновления от другого объекта (паттерн издатель–подписчик).
- **Mediator (Посредник)**: централизует сложную коммуникацию между компонентами в одном месте, снижая связанность.
- **Chain of Responsibility (Цепочка ответственности)**: передаёт запрос по цепочке обработчиков, пока один из них не обработает запрос.
---

## Command Pattern / Команда

**English — Concept**

The Command pattern turns a request into a stand-alone object that contains all information about the request. This decouples the object that invokes the operation from the one that knows how to perform it. Common uses: queues, undo/redo, logging, transactional operations.

**Russian — Концепция**

Паттерн Команда превращает запрос в самостоятельный объект, который содержит всю информацию о запросе. Это разделяет объект, который вызывает операцию, и тот, кто её выполняет. Типичные применения: очереди, отмена/повтор, логирование, транзакции.


**English:**  
In this example, we create commands (`WriteCommand`) that know how to perform and undo an action on a `Document`.  
The `Editor` acts as the invoker — it executes and stores commands, allowing undo operations.  
This separates the logic of performing actions from the logic of managing them.

**Русский:**  
В этом примере создаются команды (`WriteCommand`), которые знают, как выполнить и отменить действие над объектом `Document`.  
`Editor` выступает как вызывающий объект — он выполняет и сохраняет команды, что позволяет отменять действия.  
Это разделяет логику выполнения действий и управления ими.


In [1]:
# Command pattern — simple example: Text editor commands with undo
from abc import ABC, abstractmethod

class Command(ABC):
    @abstractmethod
    def execute(self): pass
    @abstractmethod
    def undo(self): pass

# Receiver
class Document:
    def __init__(self):
        self.text = ""

    def write(self, s):
        self.text += s

    def erase(self, count):
        if count > 0:
            self.text = self.text[:-count]

# Concrete commands
class WriteCommand(Command):
    def __init__(self, doc, text):
        self.doc = doc
        self.text = text

    def execute(self):
        self.doc.write(self.text)

    def undo(self):
        self.doc.erase(len(self.text))

# Invoker
class Editor:
    def __init__(self):
        self.history = []

    def do(self, command: Command):
        command.execute()
        self.history.append(command)

    def undo(self):
        if self.history:
            cmd = self.history.pop()
            cmd.undo()

# Demo
doc = Document()
editor = Editor()

cmd1 = WriteCommand(doc, "Hello")
editor.do(cmd1)
cmd2 = WriteCommand(doc, " World")
editor.do(cmd2)

print("After writes:", doc.text)  # English: After writes: Hello World
editor.undo()
print("After undo:", doc.text)    # English: After undo: Hello

# Russian comments:
# Демонстрация: выполняем две команды записи, затем отменяем последнюю.



After writes: Hello World
After undo: Hello


## Observer Pattern / Наблюдатель

**English — Concept**

Observer (publish–subscribe) defines a one-to-many dependency so that when one object changes state, all its dependents are notified and updated automatically. Useful for GUIs, event systems, and reactive programming.

**Russian — Концепция**

Наблюдатель (издатель–подписчик) определяет зависимость «один-ко-многим», чтобы при изменении состояния одного объекта все зависимые объекты автоматически уведомлялись и обновлялись. Полезно для GUI, систем событий и реактивного программирования.


**English:**  
`WeatherStation` keeps a list of observers (subscribers) and notifies them when its state (temperature) changes.  
Each `Display` object implements an `update()` method that reacts to these changes.  
This allows loose coupling — the station doesn’t need to know how each observer works.

**Русский:**  
`WeatherStation` хранит список наблюдателей (подписчиков) и уведомляет их при изменении состояния (температуры).  
Каждый объект `Display` реализует метод `update()`, который реагирует на изменения.  
Это обеспечивает слабую связанность — станция не знает, как именно работает каждый наблюдатель.


In [2]:
# Observer pattern — simple example: Weather station notifying observers
from typing import List, Protocol

class Observer(Protocol):
    def update(self, temp: float): ...

class WeatherStation:
    def __init__(self):
        self._observers: List[Observer] = []
        self._temp = None

    def subscribe(self, obs: Observer):
        self._observers.append(obs)

    def unsubscribe(self, obs: Observer):
        self._observers.remove(obs)

    def set_temp(self, value: float):
        self._temp = value
        for obs in list(self._observers):
            obs.update(value)

# Concrete observers
class Display:
    def __init__(self, name):
        self.name = name

    def update(self, temp: float):
        print(f"[{self.name}] Temperature updated to {temp}°C")

ws = WeatherStation()
d1 = Display("Lobby")
d2 = Display("ServerRoom")
ws.subscribe(d1)
ws.subscribe(d2)

ws.set_temp(22.5)
ws.set_temp(23.0)

# Russian comments:
# Пример: метеостанция оповещает несколько дисплеев об изменении температуры.


[Lobby] Temperature updated to 22.5°C
[ServerRoom] Temperature updated to 22.5°C
[Lobby] Temperature updated to 23.0°C
[ServerRoom] Temperature updated to 23.0°C


## Mediator Pattern / Посредник

**English — Concept**

Mediator centralizes complex interactions between multiple objects by having them communicate through a single mediator object. This reduces direct references between colleagues and lowers coupling.

**Russian — Концепция**

Посредник централизует сложные взаимодействия между объектами, заставляя их общаться через единый объект-посредник. Это уменьшает прямые ссылки между компонентами и снижает связанность.


**English:**  
The `ChatRoom` acts as a mediator — all participants communicate through it instead of directly with each other.  
When one participant sends a message, the mediator delivers it to all others.  
This reduces coupling between participants and centralizes communication.

**Русский:**  
`ChatRoom` выступает как посредник — все участники общаются через него, а не напрямую друг с другом.  
Когда один участник отправляет сообщение, посредник доставляет его всем остальным.  
Это снижает связанность между участниками и централизует коммуникацию.


In [3]:
# Mediator pattern — simple chatroom mediator
from abc import ABC, abstractmethod

class Mediator(ABC):
    @abstractmethod
    def send(self, message: str, sender): pass

class ChatRoom(Mediator):
    def __init__(self):
        self.participants = []

    def register(self, participant):
        self.participants.append(participant)
        participant.mediator = self

    def send(self, message: str, sender):
        for p in self.participants:
            if p is not sender:
                p.receive(message, sender)

class Participant:
    def __init__(self, name):
        self.name = name
        self.mediator: Mediator | None = None

    def send(self, message: str):
        if self.mediator:
            self.mediator.send(message, self)

    def receive(self, message: str, sender):
        print(f"{self.name} received from {sender.name}: {message}")

# Demo
room = ChatRoom()
alice = Participant("Alice")
bob = Participant("Bob")
room.register(alice)
room.register(bob)

alice.send("Hi Bob!")
bob.send("Hello Alice!")

# Russian comments:
# Пример: чат-комната как посредник между участниками.


Bob received from Alice: Hi Bob!
Alice received from Bob: Hello Alice!


## Chain of Responsibility Pattern / Цепочка обязанностей

**English — Concept**

Chain of Responsibility passes a request along a chain of handlers. Each handler can either process the request or pass it to the next handler. Useful for layered validation, logging, or pipelines.

**Russian — Концепция**

Цепочка ответственности передаёт запрос по цепочке обработчиков. Каждый обработчик либо обрабатывает запрос, либо передаёт его дальше. Полезно для пошаговой валидации, логирования или конвейеров обработки.


**English:**  
The request passes through a chain of handlers (`AuthHandler → ValidationHandler → ProcessingHandler`).  
Each handler checks if it can handle the request; if not, it passes it to the next one.  
This allows flexible request processing and avoids hardcoded conditional logic.

**Русский:**  
Запрос проходит через цепочку обработчиков (`AuthHandler → ValidationHandler → ProcessingHandler`).  
Каждый обработчик проверяет, может ли он обработать запрос; если нет — передаёт его дальше.  
Это позволяет гибко обрабатывать запросы без жёстких условий в коде.


In [4]:
# Chain of Responsibility — simple example: request handling with multiple handlers
from abc import ABC, abstractmethod

class Handler(ABC):
    def __init__(self, successor=None):
        self._successor = successor

    @abstractmethod
    def handle(self, request): pass

class AuthHandler(Handler):
    def handle(self, request):
        if not request.get("user"):
            return "Auth failed"
        if self._successor:
            return self._successor.handle(request)
        return "Auth passed"

class ValidationHandler(Handler):
    def handle(self, request):
        if "data" not in request:
            return "Validation failed"
        if self._successor:
            return self._successor.handle(request)
        return "Validation passed"

class ProcessingHandler(Handler):
    def handle(self, request):
        # process request
        return f"Processed data: {request.get('data')}"

# Build chain: Auth -> Validation -> Processing
chain = AuthHandler(ValidationHandler(ProcessingHandler()))

print(chain.handle({"user": "alice", "data": "payload"}))  # processed
print(chain.handle({"data": "payload"}))                  # auth failed
print(chain.handle({"user": "bob"}))                      # validation failed

# Russian comments:
# Пример: цепочка обработчиков для аутентификации, валидации и обработки.


Processed data: payload
Auth failed
Validation failed


### 📋 Pattern Comparison Overview / Обзор сравнения паттернов

**English:**  
The table below summarizes the key similarities and differences between four orchestration patterns — Command, Observer, Mediator, and Chain of Responsibility.  
It highlights their main ideas, participants, communication structures, and typical use cases.

**Русский:**  
В таблице ниже приведено сравнение четырёх паттернов оркестровки — Команда, Наблюдатель, Посредник и Цепочка ответственности.  
Показаны их основные идеи, участники, типы взаимодействия и типичные области применения.


### 🧩 Comparison Table — Orchestration Patterns

| **Pattern** | **Main Idea (English)** | **Main Idea (Русский)** | **Key Participants** | **Communication Type** | **Typical Use Cases** |
|--------------|-------------------------|--------------------------|----------------------|------------------------|-----------------------|
| **Command** | Encapsulate a request as an object, allowing to parameterize clients, queue, log, or undo operations. | Инкапсулирует запрос как объект, что позволяет параметризовать клиентов, ставить команды в очередь, логировать или отменять операции. | Invoker, Command, Receiver | One-to-one (Invoker → Command → Receiver) | Undo/Redo, Queues, Macro commands |
| **Observer** | Define a one-to-many dependency between objects so that when one changes, others are notified. | Определяет зависимость «один-ко-многим» между объектами: при изменении одного другие уведомляются. | Subject, Observer(s) | One-to-many (Publisher → Subscribers) | Event systems, GUIs, Data binding |
| **Mediator** | Centralize complex communication between multiple objects using a mediator object. | Централизует взаимодействие между несколькими объектами через посредника. | Mediator, Colleague(s) | Many-to-many (through Mediator) | Chat systems, GUI dialogs, Message routing |
| **Chain of Responsibility** | Pass a request along a chain of handlers until one handles it. | Передаёт запрос по цепочке обработчиков, пока один не обработает его. | Handler, Concrete Handlers | Sequential (Handler1 → Handler2 → …) | Validation pipelines, Logging, Request filtering |


## Summary / Краткое содержание

**English**

- These patterns help structure control flow and communication in applications.
- Command: encapsulate requests; good for undo and queuing.
- Observer: decouple publishers and subscribers for event-driven systems.
- Mediator: centralize interactions to reduce coupling.
- Chain of Responsibility: chain handlers to process requests step-by-step.

**Russian (Русский)**

- Эти паттерны помогают структурировать управление и коммуникацию в приложениях.
- Command: инкапсулирует запросы; полезен для отмены и очередей.
- Observer: разделяет издателей и подписчиков для событийных систем.
- Mediator: централизует взаимодействие, снижая связанность.
- Chain of Responsibility: цепочка обработчиков для пошаговой обработки запросов.

---


## 📚 References / Список литературы

**English Sources:**
1. Gamma, E., Helm, R., Johnson, R., & Vlissides, J. (1994). *Design Patterns: Elements of Reusable Object-Oriented Software.* Addison-Wesley.  
   → The classic “Gang of Four” book introducing Command, Observer, Mediator, and Chain of Responsibility patterns.
2. Freeman, E., Freeman, E., Bates, B., & Sierra, K. (2004). *Head First Design Patterns.* O'Reilly Media.  
   → A beginner-friendly, visual explanation of design patterns with examples in Java and Python.
3. Refactoring.Guru — *Design Patterns Explained Simply.*  
   [https://refactoring.guru/design-patterns](https://refactoring.guru/design-patterns)  
   → Excellent resource with clear illustrations and Python examples.

**Russian Sources / Русскоязычные источники:**
1. Гамма Э., Хелм Р., Джонсон Р., Влиссидес Дж. *Приемы объектно-ориентированного проектирования. Паттерны проектирования.* — СПб: Питер, 2021.  
   → Русский перевод классической книги “Gang of Four”.
2. Refactoring.Guru (на русском):  
   [https://refactoring.guru/ru/design-patterns](https://refactoring.guru/ru/design-patterns)  
   → Очень доступные объяснения паттернов с примерами кода.
3. Хабр — статьи по паттернам проектирования:  
   [https://habr.com/ru/search/?q=паттерны%20проектирования](https://habr.com/ru/search/?q=паттерны%20проектирования)  
   → Обзоры, примеры и обсуждения для практикующих программистов.

---

